In [ ]:
import numpy as np
import pandas as pd
import tracktor as tr
import cv2
import sys
from sklearn.cluster import KMeans
from scipy.optimize import linear_sum_assignment
from scipy.spatial.distance import cdist

## Global parameters
This cell (below) enlists user-defined parameters

In [ ]:
# colours is a vector of BGR values which are used to identify individuals in the video
# t_id is termite id and is also used for individual identification
# number of elements in colours should be greater than n_inds (THIS IS NECESSARY FOR VISUALISATION ONLY)
# number of elements in t_id should be greater than n_inds (THIS IS NECESSARY TO GET INDIVIDUAL-SPECIFIC DATA)
n_inds = 9
t_id = map(chr, range(65, 123))[:n_inds+8]
colours = [list(np.random.choice(range(256), size=3)) for i in range(n_inds+8)]

# this is the block_size and offset used for adaptive thresholding (block_size should always be odd)
# these values are critical for tracking performance
block_size = 51
offset = 10

# the scaling parameter can be used to speed up tracking if video resolution is too high (use value 0-1)
scaling = 1.0

# minimum area and maximum area occupied by the animal in number of pixels
# this parameter is used to get rid of other objects in view that might be hard to threshold out but are differently sized
min_area = 2
max_area = 10

# mot determines whether the tracker is being used in noisy conditions to track a single object or for multi-object
# using this will enable k-means clustering to force n_inds number of animals
mot = False

# in this example, we use a circular mask to ignore area outside the petri-dish
# mask_offset represents offset of circle from centre of the frame
mask_offset_x = -5
mask_offset_y = -5

# name of source video and paths
video = 'Ctest2'
input_vidpath = 'out_test/crop/' + video + '.mp4'
output_vidpath = 'out_test/result/' + video + '_tracktor.mp4'
output_filepath = 'out_test/' + video + '_tracktor.csv'
codec = 'DIVX' # try other codecs if the default doesn't work ('DIVX', 'avc1', 'XVID') note: this list is non-exhaustive

The cell below runs the tracking code

In [ ]:
## Open video
cap = cv2.VideoCapture(input_vidpath)
if cap.isOpened() == False:
    sys.exit('Video file cannot be read! Please check input_vidpath to ensure it is correctly pointing to the video file')

## Video writer class to output video with contour and centroid of tracked object(s)
# make sure the frame size matches size of array 'final'
fourcc = cv2.VideoWriter_fourcc(*codec)
output_framesize = (int(cap.read()[1].shape[1]*scaling),int(cap.read()[1].shape[0]*scaling))
out = cv2.VideoWriter(filename = output_vidpath, fourcc = fourcc, fps = 60.0, frameSize = output_framesize, isColor = True)

## Individual location(s) measured in the last and current step
meas_last = list(np.zeros((n_inds,2)))
meas_now = list(np.zeros((n_inds,2)))

df = []
last = 0

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    this = cap.get(1)
    if ret == True:
        # Preprocess the image for background subtraction
        frame = cv2.resize(frame, None, fx = scaling, fy = scaling, interpolation = cv2.INTER_LINEAR)
        
        # Apply mask to ignore area outside the petri dish
        mask = np.zeros((frame.shape[0], frame.shape[1]))
        cv2.circle(mask, (mask.shape[1]//2 + mask_offset_x, mask.shape[0]//2 + mask_offset_y), 520, 255, -1)
        frame[mask == 0] = 0
        
        thresh = tr.colour_to_thresh(frame, block_size, offset)
        # Custom background subtraction
        bg = thresh.copy()
        cv2.circle(bg, (bg.shape[1]//2 + mask_offset_x, bg.shape[0]//2 + mask_offset_y), 520, 0, -1)
        bgsub = thresh - bg
        
        final, contours, meas_last, meas_now = tr.detect_and_draw_contours(frame, thresh, meas_last, meas_now, min_area, max_area)
        if len(meas_now) != n_inds:
            contours, meas_now = tr.apply_k_means(contours, n_inds, meas_now)
        
        row_ind, col_ind = tr.hungarian_algorithm(meas_last, meas_now)
        final, meas_now, df = tr.reorder_and_draw(final, colours, n_inds, col_ind, meas_now, df, mot, this)
        
        # Create output dataframe
        for i in range(n_inds):
            df.append([this, meas_now[i][0], meas_now[i][1], t_id[i]])
        
        # Display the resulting frame
        out.write(final)
        cv2.imshow('frame', final)
        if cv2.waitKey(1) == 27:
            break
            
    if last >= this:
        break
    
    last = this

## Write positions to file
df = pd.DataFrame(np.matrix(df), columns = ['frame','pos_x','pos_y', 'id'])
df.to_csv(output_filepath, sep=',')

## When everything done, release the capture
cap.release()
out.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

## Summary statistics
The cells below provide functions to perform basic summary statistics - in this case, group polarisation and rotation around group centroid and rotation around centre of the arena. Polarisation measures how aligned individuals in a group are whereas rotation is a measure of the group being in a milling state aka. rotating. Checkout Tunstrom et al. 2013 PLoS Computational Biology for a more detailed description.

#### Group showing high polarisation &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; Group transitioning from high rotation to high polarisation

<img src="imgs/pol.png", width="48%", height="48%", style="float: left">

<img src="imgs/rot2pol.png", width="48%", height="48%", style="float: right">

In [ ]:
df = pd.read_csv(output_filepath)
df.head()

In [ ]:
df = df.drop(columns=['Unnamed: 0'])
df['dir_x'] = df['pos_x'] - df['pos_x'].shift(n_inds)
df['dir_y'] = df['pos_y'] - df['pos_y'].shift(n_inds)
df['speed'] = np.sqrt(df['dir_x']**2 + df['dir_y']**2)
df['dir_x'] /= df['speed']
df['dir_y'] /= df['speed']

# drop any inf in data
df = df.replace([np.inf, -np.inf], np.nan).dropna(axis=0, how="any")

df.head()

In [ ]:
for idx, ID in enumerate(np.unique(df['id'])):
    df['id'][df['id'] == ID] = idx

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8,8))
plt.scatter(df['pos_x'], df['pos_y'], c=df['id'], cmap='jet', alpha=0.5)
plt.xlabel('X', fontsize=16)
plt.ylabel('Y', fontsize=16)
plt.tight_layout()
plt.savefig('imgs/ex4_fig1.eps', format='eps', dpi=300)
plt.show()

In [ ]:
df_ord = df.groupby(['frame']).mean()
df_ord = df_ord.reset_index()

df_ord = df_ord.rename(index=str, columns={'pos_x':'centroid_x', 'pos_y':'centroid_y', 'dir_x':'gdir_x', 'dir_y':'gdir_y', 'speed':'gspeed'})

In [ ]:
df = pd.merge(df, df_ord, how='left')

In [ ]:
tmpx = (df['pos_x'] - df['centroid_x'])
tmpy = (df['pos_y'] - df['centroid_y'])
df['centroid_disp_x'] = tmpx / np.linalg.norm((tmpx, tmpy), axis=0)
df['centroid_disp_y'] = tmpy / np.linalg.norm((tmpx, tmpy), axis=0)

tmpx = (df['pos_x'] - mask.shape[1]//2 + mask_offset_x)
tmpy = (df['pos_x'] - mask.shape[0]//2 + mask_offset_y)
df['arena_centre_disp_x'] = tmpx / np.linalg.norm((tmpx, tmpy), axis=0)
df['arena_centre_disp_y'] = tmpy / np.linalg.norm((tmpx, tmpy), axis=0)

In [ ]:
centroid_disp = [df['centroid_disp_x'], df['centroid_disp_y']]
direction = [df['dir_x'], df['dir_y']]

arena_disp = [df['arena_centre_disp_x'], df['arena_centre_disp_y']]

In [ ]:
df['rotation'] = np.cross(np.array(centroid_disp).T, np.array(direction).T)
df['arena_rotation'] = np.cross(np.array(arena_disp).T, np.array(direction).T)

In [ ]:
df_ord = df.groupby(['frame']).agg(['sum', 'count'])
df_ord = df_ord.reset_index()

In [ ]:
df_ord['polarisation'] = np.sqrt(df_ord['dir_x']['sum']**2 + df_ord['dir_y']['sum']**2) / df_ord['dir_x']['count']
df_ord['rotation'] = df_ord['rotation']['sum'].divide(df_ord['rotation']['count'])
df_ord['arena_rotation'] = df_ord['arena_rotation']['sum'].divide(df_ord['arena_rotation']['count'])

In [ ]:
plt.figure(figsize=(30,3))
plt.plot(df_ord['frame']/30, df_ord['rotation']['sum'], color='r', label='Rotatioin (about group centroid)')
plt.plot(df_ord['frame']/30, df_ord['arena_rotation']['sum'], color='g', label='Rotation (about arena centre)') 
plt.plot(df_ord['frame']/30, df_ord['polarisation'], color='b', label='Polarisation')
plt.xlabel('Time (s)', fontsize=16)
plt.legend(loc=1)
plt.tight_layout()
plt.savefig('imgs/ex4_fig2.eps', format='eps', dpi=300)
plt.show()

In [ ]:
import seaborn as sns

plot = sns.jointplot(df_ord['polarisation'], df_ord['rotation']['sum'], xlim=[0,1], ylim=[-1,1], kind="hex", space=0, size = 7.5, stat_func=None, ratio=3)
plot.set_axis_labels('Polarisation', 'Rotation (about group centroid)', fontsize=16)
plt.tight_layout()
plt.savefig('imgs/ex4_fig3a.eps', format='eps', dpi=300)
plt.show()

In [ ]:
plot = sns.jointplot(df_ord['polarisation'], df_ord['arena_rotation']['sum'], xlim=[0,1], ylim=[-1,1], kind="hex", space=0, size = 7.5, stat_func=None, ratio=3)
plot.set_axis_labels('Polarisation', 'Rotation (about arena centre)', fontsize=16)
plt.tight_layout()
plt.savefig('imgs/ex4_fig3b.eps', format='eps', dpi=300)
plt.show()

In [ ]:
print(np.median(df_ord['polarisation']), np.median(np.abs(df_ord['rotation']['sum'])), np.median(np.abs(df_ord['arena_rotation']['sum'])))